In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
final_infec_cormor=pd.read_csv('final_infec_cormor.csv')
final_infec_cormor['AGE'].loc[final_infec_cormor['AGE']>=300]=90

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
# drop time and postoperative features:
final_infec_cormor.drop(['EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS','HAS_CHARTEVENTS_DATA', 'DOB', 'DOD', 'DOD_HOSP', 
                         'DOD_SSN','EXPIRE_FLAG', 'ICUSTAY_ADMIT_N','ADMITTIME', 'DISCHTIME', 'DEATHTIME','INTIME', 'OUTTIME',
                         'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'WeightUnit', 'Weight_mean','Weight_min', 'Weight_max',
                         'Weight_std', 'Weight_N', 'HeightUnit','Height_mean', 'Height_min', 'Height_max', 
                         'Height_std', 'Height_N','BMI','BaselinePainScore_mean','BaselinePainScore_max', 
                         'BaselinePainScore_min','BaselinePainScore_std', 'BaselinePainScore_N','LOS(Day)',
                         'infection','Morbidities_N','PainScore_mean','RELIGION','DISCHARGE_LOCATION', 'LANGUAGE',
                         'MORTARLITY','PainScore_max','PainScore_min','PainScore_std','PainScore_count',
                         'Cormor_Peripheral Vascular Disease', 'Cormor_Valvular Disease','Cormor_Weight Loss'],
                         axis=1, inplace=True)

In [4]:
#drop na
final_infec_cormor2=final_infec_cormor.dropna()
final_infec_cormor2

,SUBJECT_ID,HADM_ID,AGE,INSURANCE,GENDER,ETHNICITY,MARITAL_STATUS,ADMISSION_TYPE,ADMISSION_LOCATION,Surgery,...,Cormor_Coagulopathy,Cormor_Congestive Heart Failure,Cormor_Depression,Cormor_Diabetes,Cormor_Fluid and electrolyte disorders,Cormor_Hypertension,Cormor_Liver Disease,Cormor_Metastatic Carcinoma,Cormor_Myocardial Infarction (heart attack),Cormor_Obesity
0,23,152223,71,Medicare/Government,M,WHITE,MARRIED,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Cardiac,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,23,124321,75,Medicare/Government,M,WHITE,MARRIED,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Neurologic,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,28,162569,74,Medicare/Government,M,WHITE,MARRIED,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Cardiac,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,32,175413,46,Medicaid,M,WHITE,Other,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Thoracic,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,35,166707,69,Medicare/Government,F,WHITE,DIVORCED/SEPARATED,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Cardiac,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20059,98759,175386,69,Self-Pay/Private,F,WHITE,WIDOWED,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Neurologic,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
20060,98761,182540,47,Medicare/Government,F,WHITE,MARRIED,ELECTIVE,PHYS REFERRAL/NORMAL DELI,General,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20061,98769,141860,80,Medicare/Government,F,WHITE,WIDOWED,EMERGENCY,CLINIC REFERRAL/PREMATURE,General,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
20063,98794,190603,78,Medicare/Government,M,WHITE,MARRIED,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Cardiac,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
final_infec_cormor2['post_infection'].value_counts()

0.0    14186
1.0     5056
Name: post_infection, dtype: int64

In [5]:
final_infec_cormor2.to_csv('final_nodummy.csv',index=False)

In [10]:
#categorical var
category=final_infec_cormor2[['SUBJECT_ID','HADM_ID','INSURANCE','GENDER','ETHNICITY','MARITAL_STATUS','ADMISSION_TYPE',
                             'ADMISSION_LOCATION','Surgery']]
#continuous var
continuous=final_infec_cormor2[['SUBJECT_ID','HADM_ID','AGE']]
#dummy var
dummies=final_infec_cormor2[['SUBJECT_ID','HADM_ID','INSURANCE','GENDER','ETHNICITY','MARITAL_STATUS','ADMISSION_TYPE','ADMISSION_LOCATION','Surgery','post_infection','pre_infection',
'Cormor_Alcohol or drug abuse','Cormor_Cancer','Cormor_Cerebrovascular Disease (stroke)','Cormor_Chronic Kidney Disease','Cormor_Chronic Pulmonary Disease',
'Cormor_Chronic anemia','Cormor_Coagulopathy','Cormor_Congestive Heart Failure','Cormor_Depression','Cormor_Diabetes','Cormor_Fluid and electrolyte disorders',
'Cormor_Hypertension','Cormor_Liver Disease','Cormor_Metastatic Carcinoma','Cormor_Myocardial Infarction (heart attack)','Cormor_Obesity']]

In [11]:
#Get dummies
dummies=pd.get_dummies(dummies, columns=['INSURANCE','GENDER','ETHNICITY','MARITAL_STATUS','ADMISSION_TYPE','ADMISSION_LOCATION','Surgery'])
dummies[['pre_infection','post_infection','Cormor_Alcohol or drug abuse','Cormor_Cancer','Cormor_Cerebrovascular Disease (stroke)','Cormor_Chronic Kidney Disease','Cormor_Chronic Pulmonary Disease',
'Cormor_Chronic anemia','Cormor_Coagulopathy','Cormor_Congestive Heart Failure','Cormor_Depression','Cormor_Diabetes','Cormor_Fluid and electrolyte disorders',
'Cormor_Hypertension','Cormor_Liver Disease','Cormor_Metastatic Carcinoma','Cormor_Myocardial Infarction (heart attack)','Cormor_Obesity']]=dummies[['pre_infection','post_infection','Cormor_Alcohol or drug abuse','Cormor_Cancer','Cormor_Cerebrovascular Disease (stroke)','Cormor_Chronic Kidney Disease','Cormor_Chronic Pulmonary Disease',
'Cormor_Chronic anemia','Cormor_Coagulopathy','Cormor_Congestive Heart Failure','Cormor_Depression','Cormor_Diabetes','Cormor_Fluid and electrolyte disorders',
'Cormor_Hypertension','Cormor_Liver Disease','Cormor_Metastatic Carcinoma','Cormor_Myocardial Infarction (heart attack)','Cormor_Obesity']].astype('category')
dummies.to_csv('dummies2.csv', index=False)
#reference group
dummies.drop(['INSURANCE_Medicare/Government','GENDER_M','ETHNICITY_WHITE','MARITAL_STATUS_MARRIED','ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI','Surgery_Cardiac','ADMISSION_TYPE_EMERGENCY'], axis=1, inplace=True)

In [41]:
dummies

,SUBJECT_ID,HADM_ID,post_infection,pre_infection,Cormor_Alcohol or drug abuse,Cormor_Cancer,Cormor_Cerebrovascular Disease (stroke),Cormor_Chronic Kidney Disease,Cormor_Chronic Pulmonary Disease,Cormor_Chronic anemia,...,ADMISSION_LOCATION_CLINIC REFERRAL/PREMATURE,ADMISSION_LOCATION_EMERGENCY ROOM ADMIT,ADMISSION_LOCATION_Other,ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM,Surgery_Circulatory,Surgery_General,Surgery_Musculoskeletal,Surgery_Neurologic,Surgery_Plastic,Surgery_Thoracic
0,23,152223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,23,124321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
2,28,162569,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,32,175413,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,35,166707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20059,98759,175386,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,1,0,0
20060,98761,182540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
20061,98769,141860,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1,0,0,0,0,1,0,0,0,0
20063,98794,190603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Apply normalization
cols_to_normalize = ['AGE']
continuous_norm=copy.deepcopy(continuous)
for col in cols_to_normalize:
    continuous_norm[col] = (continuous[col] - continuous[col].mean()) / continuous[col].std()

In [13]:
final_R=pd.merge(dummies,continuous_norm,how='inner')
final_R.columns = ['SUBJECT_ID', 'HADM_ID', 'post_infection', 'pre_infection',
       'Alcohol or drug abuse', 'Cancer',
       'Cerebrovascular Disease (stroke)',
       'Chronic Kidney Disease', 'Chronic Pulmonary Disease',
       'Chronic anemia', 'Coagulopathy',
       'Congestive Heart Failure', 'Depression',
       'Diabetes', 'Fluid and electrolyte disorders',
       'Hypertension', 'Liver Disease',
       'Metastatic Carcinoma',
       'Myocardial Infarction (heart attack)', 'Obesity',
       'INSURANCE_Medicaid',
       'INSURANCE_Self-Pay/Private', 'GENDER_F',
       'ETHNICITY_BLACK/AFRICAN', 'ETHNICITY_HISPANIC/LATINO',
       'ETHNICITY_OTHER SPECIES', 'ETHNICITY_UNKNOWN',
       'MARITAL_STATUS_DIVORCED/SEPARATED', 'MARITAL_STATUS_Other',
       'MARITAL_STATUS_SINGLE', 'MARITAL_STATUS_WIDOWED',
       'ADMISSION_TYPE_ELECTIVE', 'ADMISSION_TYPE_URGENT',
       'ADMISSION_LOCATION_CLINIC REFERRAL/PREMATURE',
       'ADMISSION_LOCATION_EMERGENCY ROOM ADMIT', 'ADMISSION_LOCATION_Others',
       'ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM', 'Surgery_Circulatory',
       'Surgery_General', 'Surgery_Musculoskeletal', 'Surgery_Neurologic',
       'Surgery_Plastic', 'Surgery_Thoracic', 'AGE']

In [14]:
#retain 1 record of each patient
final_R_0423 = final_R.groupby('SUBJECT_ID').sample(n=1).reset_index(drop=True)
#final_R_0423.to_csv('final_R_0423.csv', index=False)

In [15]:
final_R_0423

,SUBJECT_ID,HADM_ID,post_infection,pre_infection,Alcohol or drug abuse,Cancer,Cerebrovascular Disease (stroke),Chronic Kidney Disease,Chronic Pulmonary Disease,Chronic anemia,...,ADMISSION_LOCATION_EMERGENCY ROOM ADMIT,ADMISSION_LOCATION_Others,ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM,Surgery_Circulatory,Surgery_General,Surgery_Musculoskeletal,Surgery_Neurologic,Surgery_Plastic,Surgery_Thoracic,AGE
0,3,145834,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,0,0,0,0,0,0.839660
1,6,107064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,1,0,0,0,0,0.123426
2,11,194540,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,0,-0.918368
3,12,112213,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0.514099
4,13,143045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,-1.569490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17013,99957,148574,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,0,0,0.253650
17014,99982,151454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0.058314
17015,99991,151118,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,-1.048592
17016,99995,137810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1.621005


In [16]:
final_R_0423['post_infection'].value_counts()

0.0    12735
1.0     4283
Name: post_infection, dtype: int64